# Bi-linear interaction model with group-by augmentations

## Prepare dataset for training

Following the same steps as in [the training for Movielens/IMDB dataset](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/movielens%20simple%20model.ipynb), we first load splitted dataset generated in [notebook](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/dataset_preprocessing/movielens%20with%20imdb.ipynb)

In [1]:
DATASET = 'rees_ecommerce'

In [2]:
from utils import load_dataset

datasets = {}
for split_name in ['train', 'val', 'test']:
    datasets[split_name] = load_dataset(DATASET, split_name)

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


Then we parse features' names to obtain a list of offer features (that will be used to modelize film) and a list of user features (aggregated history up to chosen date)

In [3]:
from utils import AGG_PREFIX

all_columns = list(datasets['train'].element_spec.keys())
technical_columns = ['user_id', 'date']
user_features = list(filter(lambda x: x.startswith(AGG_PREFIX), all_columns))
offer_features = list(filter(lambda x: x not in user_features + technical_columns, all_columns))

### Rebatching datasets

Splitting dataset into smaller batches in the same way as described in [the training for Movielens/IMDB dataset](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/movielens%20simple%20model.ipynb)

In [4]:
%%time

from functools import partial
from uuid import uuid4

from utils import rebatch_by_events

datasets['train'] = rebatch_by_events(datasets['train'], batch_size=5040, date_column='date', nb_events_by_user_by_day=8)
for key in ['val', 'test']:
    datasets[key] = rebatch_by_events(datasets[key], batch_size=5040, date_column='date', nb_events_by_user_by_day=8,
                                      seed=1729).cache(f'/tmp/{uuid4()}.tf')

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
CPU times: user 30.8 s, sys: 1.19 s, total: 32 s
Wall time: 16.1 s


In [5]:
from utils import add_equal_weights

for key in datasets:
    datasets[key] = datasets[key].map(partial(add_equal_weights, features=offer_features))

## Define the model

First we need to get number of different modalities inputs can take from saved vectorizers (it will be used in embeddings layer definition):

In [6]:
from utils import load_inverse_lookups
inverse_lookups = load_inverse_lookups(DATASET)

In [7]:
import re

vocabulary_sizes = {}

for feature in offer_features:
    vocabulary_sizes[feature] = inverse_lookups[feature].vocabulary_size()

for feature in user_features:
    for key in inverse_lookups:
        pattern = re.compile(r"{}(\w+)_{}".format(AGG_PREFIX, key))
        if pattern.match(feature):
            vocabulary_sizes[feature] = vocabulary_sizes[key]

### Model architecture

In [8]:
import tensorflow as tf

Now we can assemble all these layers into final model. Note that offer compression weights and interaction kernels are shared between different augmentations we generate.

<img src="resources/group_by_augmentations_model.png" alt="model" width="800" />

### Model parameters

In [9]:
EMBEDDING_DIM = 100
L1_COEFF = 2e-7
DROPOUT = 0.1
NB_AUGMENTATIONS = 3
AVERAGE_NUMBER_OF_FEATURES_IN_AUGMENTATION = 2
USER_META_FEATURES = 5
OFFER_META_FEATURES = 3

def REGULARIZER():
    return {'class_name': 'L1L2', 'config': {'l1': L1_COEFF, 'l2': 0.}}

def OUTPUT_DNN():
    return tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(100,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Activation('tanh'),
        tf.keras.layers.Dense(50,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Activation('tanh'),
        tf.keras.layers.Dense(1,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
    ], name='output_Dnn')

EPOCHS = 8

NUMBER_OF_NEGATIVES = 4
LOSS = tf.keras.losses.BinaryCrossentropy(from_logits=True)
AUC_METRIC = tf.keras.metrics.AUC(from_logits=True)

import tensorflow_addons as tfa
OPTIMIZER = tfa.optimizers.AdamW(weight_decay=4e-8, learning_rate=0.0008)

### Embeddings

We will define embeddings with the same `WeightedEmbeddings` layer described in [the training of a simple model](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/movielens%20simple%20model.ipynb) with one addition:
* for offer features when aggregating a list of embedding vectors, we will also calculate variance and not only mean vector

It is easy to do in the same sparse-dense matrix multiplication operation as mean calculation (we get second moment and then calculate variance from it).

In [10]:
from layers import get_input_layer, WeightedEmbeddings
from utils import WEIGHT_SUFFIX

inputs = {}
embedded_user_features, embedded_offer_features, variance_offer_features = {}, {}, {}
for feature in user_features:
    inputs[feature] = get_input_layer(feature)
    emb_layer = WeightedEmbeddings(vocabulary_sizes[feature],
                                   EMBEDDING_DIM, name=f'{feature}_embedding',
                                   embeddings_regularizer=REGULARIZER())
    embedded_user_features[feature] = emb_layer(inputs[feature])
for feature in offer_features:
    # for offer features we need weights:
    # with dummy weights during training, and the ones used for a feature's averaging at inference time
    inputs[f'{feature}_weight'] = get_input_layer(f'{feature}_weight', tf.float32)
    inputs[feature] = get_input_layer(feature)
    emb_layer = WeightedEmbeddings(vocabulary_sizes[feature],
                                   EMBEDDING_DIM, name=f'{feature}_embedding',
                                   embeddings_regularizer=REGULARIZER(),
                                   calculate_variance=True)
    embedded_offer_features[feature], variance_offer_features[feature] =\
        emb_layer(inputs[feature], inputs[f'{feature}_weight'])

### Combining everything into model

Now we can define described model architecture on the top of embeddings.

In [11]:
user_stacked = tf.stack(list(embedded_user_features.values()), axis=1)
offer_stacked = tf.stack(list(embedded_offer_features.values()), axis=1)
offer_variance = tf.stack(list(variance_offer_features.values()), axis=1)
stacked_raw_offer_attrs = tf.stack([tf.cast(inp.values, tf.int32) for feature, inp in inputs.items()
                                    if feature in offer_features], axis=1)

In [12]:
from layers import KeyGenerator, GroupBy
from layers import UserFeaturesCompressor
from layers import OfferFeaturesCompressor

from layers import MaskNet

from layers import BiLinearInteraction


group_by = GroupBy(name='group_by')
key_generator = KeyGenerator(number_of_offer_attributes=len(offer_features),
                             average_number_of_attributes_in_key=AVERAGE_NUMBER_OF_FEATURES_IN_AUGMENTATION,
                             name='grp_key_generator')

user_compressed = UserFeaturesCompressor(USER_META_FEATURES, DROPOUT,
                                         name='user_compressor')(user_stacked)
offer_features_compressor = OfferFeaturesCompressor(OFFER_META_FEATURES, DROPOUT, name='offer_compressor')
mask_net = MaskNet(OFFER_META_FEATURES, DROPOUT, name='mask_generation')
apply_mask = tf.keras.layers.Multiply(name='apply_mask')
bi_linear_interaction = BiLinearInteraction(number_of_negatives=NUMBER_OF_NEGATIVES, dropout_rate=DROPOUT,
                                            initializer='random_normal', regularizer=REGULARIZER(),
                                            name='interaction')
output_dnn = OUTPUT_DNN()

augmentation_predictions = []
for i in range(NB_AUGMENTATIONS):
    group_by_key = key_generator(stacked_raw_offer_attrs)
    mean_offer_emb, variance_offer_emb = group_by(group_by_key, offer_stacked)
    compressed_offer_embeddings = offer_features_compressor([mean_offer_emb, variance_offer_emb])
    mask = mask_net([mean_offer_emb, variance_offer_emb])
    masked_offer_embeddings = apply_mask([compressed_offer_embeddings, mask])
    _output = output_dnn(bi_linear_interaction([user_compressed, masked_offer_embeddings], generate_negatives=True))
    augmentation_predictions.append(_output)
output = tf.concat(augmentation_predictions, axis=1)

/usr/local/lib/python3.9/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


And for evaluation we don't need to create augmentations, we need just to take offer features' mean and variance coming from inputs.

In [13]:
compressed_offer_embeddings = offer_features_compressor([offer_stacked, offer_variance])
mask = mask_net([offer_stacked, offer_variance])
masked_offer_embeddings = apply_mask([compressed_offer_embeddings, mask])

eval_output = output_dnn(bi_linear_interaction([user_compressed, masked_offer_embeddings], generate_negatives=True))

In [14]:
from utils import BroadcastLoss, BroadcastMetric

model = tf.keras.Model(inputs, output, name='group_by_augmentations')
model.compile(optimizer=OPTIMIZER,
              loss=BroadcastLoss(LOSS, NUMBER_OF_NEGATIVES),
              metrics=[BroadcastMetric(AUC_METRIC, NUMBER_OF_NEGATIVES)])

eval_model = tf.keras.Model(inputs, eval_output, name='group_by_augmentations_eval')

### Training

In [15]:
model.fit(datasets['train'], epochs=EPOCHS, validation_data=datasets['val'])

Epoch 1/8


/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['date', 'user_id'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


679/679 [==============================] - 427s 528ms/step - loss: 0.4686 - auc: 0.6910 - val_loss: 0.4479 - val_auc: 0.7155
Epoch 2/8
679/679 [==============================] - 317s 467ms/step - loss: 0.4448 - auc: 0.7195 - val_loss: 0.4440 - val_auc: 0.7212
Epoch 3/8
679/679 [==============================] - 349s 513ms/step - loss: 0.4418 - auc: 0.7251 - val_loss: 0.4435 - val_auc: 0.7236
Epoch 4/8
679/679 [==============================] - 325s 477ms/step - loss: 0.4399 - auc: 0.7288 - val_loss: 0.4448 - val_auc: 0.7228
Epoch 5/8
679/679 [==============================] - 316s 466ms/step - loss: 0.4398 - auc: 0.7296 - val_loss: 0.4419 - val_auc: 0.7260
Epoch 6/8
679/679 [==============================] - 327s 482ms/step - loss: 0.4386 - auc: 0.7324 - val_loss: 0.4434 - val_auc: 0.7257
Epoch 7/8
679/679 [==============================] - 266s 391ms/step - loss: 0.4383 - auc: 0.7337 - val_loss: 0.4417 - val_auc: 0.7296
Epoch 8/8
679/679 [==============================] - 281s 414ms/s

## Single task models benchmark

Using same approach as in [the simple model notebook](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/movielens%20simple%20model.ipynb) we will look at performance gap between the model with group by augmentations against set of models specialized on tasks corresponding to one offer feature at time. We won't use augmentations in those baseline models, because they will be already aligned with offer we will use in evaluation afterwards.

In [16]:
# offer columns we want to evaluate, specific to dataset we test
TASKS = ['product_id', 'category1', 'category2', 'category3', 'brand', 'priceCluster']

In [17]:
def bi_linear_interaction_model(single_task_feature, name='bi_linear_model'):
    # user_features, vocabulary_sizes, EMBEDDING_DIM, REGULARIZER, USER_TOWER, OFFER_TOWER,
    # OPTIMIZER, LOSS, NUMBER_OF_NEGATIVES
    # come from global scope, but can be passed as params instead
    inputs = {}
    embedded_user_features, embedded_offer_features, variance_offer_features = {}, {}, {}
    for feature in user_features:
        inputs[feature] = get_input_layer(feature)
        emb_layer = WeightedEmbeddings(vocabulary_sizes[feature],
                                       EMBEDDING_DIM, name=f'{feature}_embedding',
                                       embeddings_regularizer=REGULARIZER())
        embedded_user_features[feature] = emb_layer(inputs[feature])

    # for offer feature we need weights:
    # with dummy weights during training, and the ones used for a feature's averaging at inference time
    inputs[f'{single_task_feature}_weight'] = get_input_layer(f'{single_task_feature}_weight', tf.float32)
    inputs[single_task_feature] = get_input_layer(single_task_feature)
    emb_layer = WeightedEmbeddings(vocabulary_sizes[single_task_feature],
                                   EMBEDDING_DIM, name=f'{single_task_feature}_embedding',
                                   embeddings_regularizer=REGULARIZER())
    embedded_offer_feature = emb_layer(inputs[single_task_feature],
                                       inputs[f'{single_task_feature}_weight'])
    
    user_stacked = tf.stack(list(embedded_user_features.values()), axis=1)
    offer_stacked = tf.expand_dims(embedded_offer_feature, axis=1)
    
    
    user_compressed = UserFeaturesCompressor(USER_META_FEATURES, DROPOUT,
                                             name='user_compressor')(user_stacked)
    mask_net = MaskNet(OFFER_META_FEATURES, DROPOUT, name='mask_generation')
    apply_mask = tf.keras.layers.Multiply(name='apply_mask')
    bi_linear_interaction = BiLinearInteraction(number_of_negatives=NUMBER_OF_NEGATIVES, dropout_rate=DROPOUT,
                                                initializer='random_normal', regularizer=REGULARIZER(),
                                                name='interaction')
    output_dnn = OUTPUT_DNN()

    
    mask = mask_net([offer_stacked, offer_stacked])
    masked_offer_embeddings = apply_mask([offer_stacked, mask])
    
    output = OUTPUT_DNN()(bi_linear_interaction([user_compressed, masked_offer_embeddings],
                                                generate_negatives=True))

    model = tf.keras.Model(inputs, output, name=name)
    model.compile(optimizer=OPTIMIZER,
                  loss=BroadcastLoss(LOSS, NUMBER_OF_NEGATIVES),
                  metrics=[BroadcastMetric(AUC_METRIC, NUMBER_OF_NEGATIVES)])
    
    return model

In [18]:
mono_feature_models = {}
for task_offer_feature in TASKS:
    mono_feature_models[task_offer_feature] =\
        bi_linear_interaction_model(task_offer_feature, name=f'{task_offer_feature}_model')
    mono_feature_models[task_offer_feature].fit(datasets['train'],
                                                epochs=EPOCHS,
                                                validation_data=datasets['val'])

Epoch 1/8


/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['category2', 'category3', 'date', 'brand', 'priceCluster', 'user_id', 'category1', 'category2_weight', 'category3_weight', 'brand_weight', 'priceCluster_weight', 'category1_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


679/679 [==============================] - 195s 251ms/step - loss: 0.4535 - auc: 0.7277 - val_loss: 0.4385 - val_auc: 0.7514
Epoch 2/8
679/679 [==============================] - 156s 229ms/step - loss: 0.4291 - auc: 0.7639 - val_loss: 0.4338 - val_auc: 0.7572
Epoch 3/8
679/679 [==============================] - 152s 223ms/step - loss: 0.4222 - auc: 0.7732 - val_loss: 0.4351 - val_auc: 0.7575
Epoch 4/8
679/679 [==============================] - 149s 220ms/step - loss: 0.4182 - auc: 0.7798 - val_loss: 0.4357 - val_auc: 0.7567
Epoch 5/8
679/679 [==============================] - 149s 219ms/step - loss: 0.4153 - auc: 0.7848 - val_loss: 0.4375 - val_auc: 0.7565
Epoch 6/8
679/679 [==============================] - 149s 219ms/step - loss: 0.4131 - auc: 0.7892 - val_loss: 0.4405 - val_auc: 0.7552
Epoch 7/8
679/679 [==============================] - 148s 218ms/step - loss: 0.4110 - auc: 0.7931 - val_loss: 0.4438 - val_auc: 0.7534
Epoch 8/8
679/679 [==============================] - 150s 221ms/s

/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['category2', 'product_id', 'category3', 'date', 'brand', 'priceCluster', 'user_id', 'category2_weight', 'product_id_weight', 'category3_weight', 'brand_weight', 'priceCluster_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


679/679 [==============================] - 188s 239ms/step - loss: 0.4770 - auc: 0.6768 - val_loss: 0.4707 - val_auc: 0.6721
Epoch 2/8
679/679 [==============================] - 160s 236ms/step - loss: 0.4700 - auc: 0.6730 - val_loss: 0.4698 - val_auc: 0.6730
Epoch 3/8
679/679 [==============================] - 177s 259ms/step - loss: 0.4689 - auc: 0.6746 - val_loss: 0.4695 - val_auc: 0.6731
Epoch 4/8
679/679 [==============================] - 164s 241ms/step - loss: 0.4681 - auc: 0.6762 - val_loss: 0.4696 - val_auc: 0.6729
Epoch 5/8
679/679 [==============================] - 151s 223ms/step - loss: 0.4673 - auc: 0.6779 - val_loss: 0.4695 - val_auc: 0.6728
Epoch 6/8
679/679 [==============================] - 149s 220ms/step - loss: 0.4666 - auc: 0.6797 - val_loss: 0.4700 - val_auc: 0.6725
Epoch 7/8
679/679 [==============================] - 149s 220ms/step - loss: 0.4659 - auc: 0.6818 - val_loss: 0.4709 - val_auc: 0.6715
Epoch 8/8
679/679 [==============================] - 152s 224ms/s

...

679/679 [==============================] - 210s 268ms/step - loss: 0.4724 - auc: 0.6718 - val_loss: 0.4637 - val_auc: 0.6817
Epoch 2/8
679/679 [==============================] - 172s 253ms/step - loss: 0.4629 - auc: 0.6823 - val_loss: 0.4630 - val_auc: 0.6831
Epoch 3/8
679/679 [==============================] - 176s 259ms/step - loss: 0.4616 - auc: 0.6845 - val_loss: 0.4626 - val_auc: 0.6835
Epoch 4/8
679/679 [==============================] - 172s 253ms/step - loss: 0.4607 - auc: 0.6861 - val_loss: 0.4625 - val_auc: 0.6836
Epoch 5/8
679/679 [==============================] - 157s 230ms/step - loss: 0.4602 - auc: 0.6874 - val_loss: 0.4623 - val_auc: 0.6839
Epoch 6/8
679/679 [==============================] - 158s 232ms/step - loss: 0.4597 - auc: 0.6888 - val_loss: 0.4628 - val_auc: 0.6838
Epoch 7/8
679/679 [==============================] - 158s 232ms/step - loss: 0.4591 - auc: 0.6907 - val_loss: 0.4634 - val_auc: 0.6833
Epoch 8/8
679/679 [==============================] - 160s 236ms/s

/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['category2', 'product_id', 'category3', 'date', 'priceCluster', 'user_id', 'category1', 'category2_weight', 'product_id_weight', 'category3_weight', 'priceCluster_weight', 'category1_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


679/679 [==============================] - 197s 249ms/step - loss: 0.4550 - auc: 0.7120 - val_loss: 0.4447 - val_auc: 0.7302
Epoch 2/8
679/679 [==============================] - 144s 212ms/step - loss: 0.4414 - auc: 0.7341 - val_loss: 0.4415 - val_auc: 0.7336
Epoch 3/8
679/679 [==============================] - 144s 212ms/step - loss: 0.4388 - auc: 0.7379 - val_loss: 0.4409 - val_auc: 0.7343
Epoch 4/8
679/679 [==============================] - 143s 211ms/step - loss: 0.4374 - auc: 0.7400 - val_loss: 0.4407 - val_auc: 0.7346
Epoch 5/8
679/679 [==============================] - 144s 211ms/step - loss: 0.4364 - auc: 0.7419 - val_loss: 0.4414 - val_auc: 0.7342
Epoch 6/8
679/679 [==============================] - 144s 211ms/step - loss: 0.4355 - auc: 0.7440 - val_loss: 0.4423 - val_auc: 0.7342
Epoch 7/8
679/679 [==============================] - 158s 233ms/step - loss: 0.4346 - auc: 0.7462 - val_loss: 0.4425 - val_auc: 0.7338
Epoch 8/8
679/679 [==============================] - 151s 222ms/s

/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['category2', 'product_id', 'category3', 'date', 'brand', 'user_id', 'category1', 'category2_weight', 'product_id_weight', 'category3_weight', 'brand_weight', 'category1_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


679/679 [==============================] - 182s 233ms/step - loss: 0.4649 - auc: 0.6908 - val_loss: 0.4555 - val_auc: 0.6980
Epoch 2/8
679/679 [==============================] - 146s 215ms/step - loss: 0.4531 - auc: 0.7010 - val_loss: 0.4538 - val_auc: 0.7009
Epoch 3/8
679/679 [==============================] - 144s 212ms/step - loss: 0.4513 - auc: 0.7047 - val_loss: 0.4534 - val_auc: 0.7025
Epoch 4/8
679/679 [==============================] - 145s 213ms/step - loss: 0.4504 - auc: 0.7071 - val_loss: 0.4536 - val_auc: 0.7024
Epoch 5/8
679/679 [==============================] - 143s 211ms/step - loss: 0.4498 - auc: 0.7090 - val_loss: 0.4547 - val_auc: 0.7027
Epoch 6/8
679/679 [==============================] - 144s 211ms/step - loss: 0.4492 - auc: 0.7110 - val_loss: 0.4543 - val_auc: 0.7023
Epoch 7/8
679/679 [==============================] - 144s 211ms/step - loss: 0.4488 - auc: 0.7130 - val_loss: 0.4550 - val_auc: 0.7027
Epoch 8/8
679/679 [==============================] - 144s 212ms/s

## Evaluation

In [19]:
%%time
from utils import prepare_single_task_dataset
test_datasets = {}
for task_offer_feature in TASKS:
    test_datasets[task_offer_feature] = \
        prepare_single_task_dataset(datasets['test'], task_offer_feature, offer_features)

CPU times: user 4min 25s, sys: 19.1 s, total: 4min 44s
Wall time: 4min 12s


In [20]:
%%time
from collections import defaultdict
from utils import evaluate_model, wAUC

aucs = defaultdict(dict)
for task_offer_feature in TASKS:
    for model_name in TASKS:
        aucs[task_offer_feature][f'MONO:{model_name}'] = \
            evaluate_model(mono_feature_models[model_name],
                           task_offer_feature, test_datasets, NUMBER_OF_NEGATIVES, inverse_lookups)
    aucs[task_offer_feature]['group_by augmentations'] = \
            evaluate_model(eval_model, task_offer_feature, test_datasets, NUMBER_OF_NEGATIVES, inverse_lookups)

/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['category2', 'product_id', 'category3', 'date', 'brand', 'priceCluster', 'user_id', 'category2_weight', 'product_id_weight', 'category3_weight', 'brand_weight', 'priceCluster_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['product_id', 'category3', 'date', 'brand', 'priceCluster', 'user_id', 'category1', 'product_id_weight', 'category3_weight', 'brand_weight', 'priceCluster_weight', 'category1_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['category2', 'product_id', 'date', 'brand', 'pric

CPU times: user 3h 29min 4s, sys: 24min 25s, total: 3h 53min 29s
Wall time: 1h 10min 22s


In [21]:
import pandas as pd
results = pd.DataFrame()
for task_name in aucs:
    for model_name in aucs[task_name]:
        w_auc = wAUC(aucs[task_name][model_name])
        results = pd.concat([results,
                             pd.Series({'wAUC': w_auc, 'offers': task_name, 'model': model_name}).to_frame().T],
                            ignore_index=True)

In [22]:
pd.pivot_table(results, 'wAUC', 'model', 'offers')\
    .rename(columns={'priceCluster': 'price'}, index={'MONO:priceCluster': 'MONO:price'})\
    .style.background_gradient(cmap='coolwarm').format(precision=3)

offers,brand,category1,category2,category3,price,product_id
model,,,,,,
MONO:brand,0.760,0.622,0.641,0.628,0.606,0.734
MONO:category1,0.647,0.734,0.728,0.710,0.575,0.661
MONO:category2,0.654,0.722,0.741,0.701,0.571,0.667
MONO:category3,0.651,0.699,0.715,0.745,0.574,0.661
MONO:price,0.657,0.583,0.590,0.602,0.705,0.723
MONO:product_id,0.733,0.676,0.686,0.680,0.652,0.772
group_by augmentations,0.760,0.721,0.739,0.740,0.695,0.773
